# Standard States and Piecewise functions

In [ ]:
import sympy as sym
import matplotlib.pyplot as plt
import numpy as np
from thermoengine import phases, model
%matplotlib inline
sym.init_printing()
import warnings
warnings.filterwarnings('ignore')

## Get the Berman (1988) heat capacity model for Quartz ... 

In [ ]:
modelDB = model.Database()
Quartz = modelDB.get_phase('Qz')

## ... and plot the function

In [ ]:
T_array = np.linspace(250.0, 1200.0, 100, endpoint=True)
Cp_array = Quartz.heat_capacity(T_array, 1000.0)
plt.plot(T_array, Cp_array)
plt.ylabel('Cp J/K-m')
plt.xlabel('T K')
plt.show()

## The Berman (1988) $C_P$ model is formulated in terms of:
- A lattice function applicable over the whole of $T$, $P$ space:

In [ ]:
T,P,Tr,Pr,k0,k1,k2,k3 = sym.symbols('T P T_r P_r k0 k1 k2 k3')
CpPr = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3
CpPr

- and a "lambda" transition function applicable over a region of $T$, $P$ space:

In [ ]:
l1,l2 = sym.symbols('l1 l2')
TlPr, Tlref = sym.symbols('T_lambda_Pr T_lambda_ref')
Cpl = sym.Piecewise((0,T<Tlref),(0,T>TlPr),(T*(l1+l2*T)**2,True))
Cpl

## Plot $C_P$, $H$, and $S$ from the lattice contribution
- ${H_T} = {H_{{T_r}}} + \int\limits_{{T_r}}^T {{C_P}dT}$
- ${S_T} = {S_{{T_r}}} + \int\limits_{{T_r}}^T {\frac{{{C_P}}}{T}dT}$

In [ ]:
H = sym.integrate(CpPr,(T,Tr,T))
S = sym.integrate(CpPr/T,(T,Tr,T))

In [ ]:
Cp_lattice = np.empty(T_array.size)
H_lattice = np.empty(T_array.size)
S_lattice = np.empty(T_array.size)
G_lattice = np.empty(T_array.size)
for ind,t in enumerate(T_array):
    Cp_lattice[ind] = CpPr.subs([(T,t),(k0,80.01),(k1,-2.403E2),(k2,-35.467E5),(k3,49.157E7)])
    H_lattice[ind] = H.subs([(T,t),(Tr,298.15),(k0,80.01),(k1,-2.403E2),(k2,-35.467E5),(k3,49.157E7)])
    S_lattice[ind] = S.subs([(T,t),(Tr,298.15),(k0,80.01),(k1,-2.403E2),(k2,-35.467E5),(k3,49.157E7)])
    G_lattice[ind] = H_lattice[ind] - t*S_lattice[ind]
plt.figure(figsize=(16,8))
plt.subplot(141)
plt.plot(T_array, Cp_lattice)
plt.ylabel('Cp J/K-m')
plt.xlabel('T K')
plt.subplot(142)
plt.plot(T_array, S_lattice)
plt.ylabel('S J/K-m')
plt.xlabel('T K')
plt.subplot(143)
plt.plot(T_array, H_lattice)
plt.ylabel('H J/m')
plt.xlabel('T K')
plt.subplot(144)
plt.plot(T_array, G_lattice)
plt.ylabel('G J/m')
plt.xlabel('T K')
plt.show()

## Plot $C_P$, $H$, and $S$ from the "lambda" contribution

In [ ]:
Hl = sym.integrate(Cpl,(T,Tr,T))
Sl = sym.integrate(Cpl/T,(T,Tr,T))

In [ ]:
Cp_lambda = np.empty(T_array.size)
S_lambda = np.empty(Cp_array.size)
H_lambda = np.empty(Cp_array.size)
G_lambda = np.empty(Cp_array.size)
for ind,t in enumerate(T_array):
    Cp_lambda[ind] = Cpl.subs([(T,t),(Tr,298.15),(l1,-9.187E-2),(l2,24.607E-5),(Tlref,373.0),(TlPr,848.0)])
    S_lambda[ind] = Sl.subs([(T,t),(Tr,298.15),(l1,-9.187E-2),(l2,24.607E-5),(Tlref,373.0),(TlPr,848.0)])
    H_lambda[ind] = Hl.subs([(T,t),(Tr,298.15),(l1,-9.187E-2),(l2,24.607E-5),(Tlref,373.0),(TlPr,848.0)])
    G_lambda[ind] = H_lambda[ind] - t*S_lambda[ind]
plt.figure(figsize=(16,8))
plt.subplot(141)
plt.plot(T_array, Cp_lambda)
plt.ylabel('Cp J/K-m')
plt.xlabel('T K')
plt.subplot(142)
plt.plot(T_array, S_lambda)
plt.ylabel('S J/K-m')
plt.xlabel('T K')
plt.subplot(143)
plt.plot(T_array, H_lambda)
plt.ylabel('H J/m')
plt.xlabel('T K')
plt.subplot(144)
plt.plot(T_array, G_lambda)
plt.ylabel('G J/m')
plt.xlabel('T K')
plt.show()

## Plot the combined lattice and "lambda" functions

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(141)
plt.plot(T_array, Cp_lattice+Cp_lambda)
plt.plot(T_array, Cp_lattice)
plt.ylabel('Cp J/K-m')
plt.xlabel('T K')
plt.subplot(142)
plt.plot(T_array, S_lattice+S_lambda)
plt.plot(T_array, S_lattice)
plt.ylabel('S J/K-m')
plt.xlabel('T K')
plt.subplot(143)
plt.plot(T_array, H_lattice+H_lambda)
plt.plot(T_array, H_lattice)
plt.ylabel('H J/m')
plt.xlabel('T K')
plt.subplot(144)
plt.plot(T_array, G_lattice+G_lambda)
plt.plot(T_array, G_lattice)
plt.ylabel('G J/m')
plt.xlabel('T K')
plt.show()

## What does a piecewise function look printed?

In [ ]:
from sympy.printing.ccode import C99CodePrinter
printer = C99CodePrinter()
print(printer.doprint(Hl-T*Sl))